## **Обзор данных**

**Данные:**

`children` — количество детей в семье

`days_employed` — общий трудовой стаж в днях

`dob_years` — возраст клиента в годах

`education` — уровень образования клиента

`education_id` — идентификатор уровня образования

`family_status` — семейное положение

`family_status_id` — идентификатор семейного положения

`gender` — пол клиента

`income_type` — тип занятости

`debt` — имел ли задолженность по возврату кредитов

`total_income` — ежемесячный доход

`purpose` — цель получения кредита

In [1]:
import pandas as pd
from pymystem3 import Mystem

In [2]:
try:
    data = (
    pd.read_csv("D:/datasets/data.csv") 
)
except FileNotFoundError:
    print("File not found.")
except pd.errors.EmptyDataError:
    print("No data")
except pd.errors.ParserError:
    print("Parse error")
except Exception:
    print("Some other exception")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Из описания данных и первых 10 строк видно, что перед тем как приступать к анализу данных, необходимо совершеть предобработку файлов, разобраться со значениями столбца `days_employed`, `education`, а также проверить датасет на возможные пропуски/отсутствие значений.**

**`total_income` необходимо перевести в целочисленные строки для более комфортного и реалистичного подсчета значений!**

## **Предобработка данных:**

### Поиск артефактов/пропусков

**Найдем столбцы с пропущенными значениями, и заполним медианным значением по столбцу:**

In [5]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
miss_days_employed = (data['days_employed'].isnull().sum() / len(data['days_employed']))
display('Пропущенные значения составляют: {:.1%}' .format(miss_days_employed))

'Пропущенные значения составляют: 10.1%'

**Также мы видим, что присутвуют отрицательные значения, чего быть не должно, заменим данные на положительные с помощью метода `abs()`.** 

**Также заменим пропуски медианным значением**

In [7]:
data['days_employed'] = data['days_employed'].abs()

**Проверим минимальное значение**

In [8]:
data['days_employed'].min()

24.14163324048118

**В столбце days_employed найдены пропуски и артефакты - в некоторых случаях стаж отрицательный, в остальных он неправдоподобно большой. Скорее всего, база данных была ошибочно заполнена, либо данные были неправильно извлечены. Заменим отрицательные значения на полодительные, и составим функцию учитывающую пол и возраст выхода на пенсию.**

In [9]:
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())

**Заполняем пропуски столбца `уровня дохода` медианным значение для соответвующего типа занятости:**

In [10]:
data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.median()))

**Среднее** - это среднее значение набора данных, а **медиана** - это центральное числовое значение набора данных. Используем медиану, т.к. среднее значение из-за разброса min и max значений может дать неоднозначную сумму:

**e.g.** 2,20,100: `медиана` будет равняться 20, а `среднее` значение = 66. В нашем случае, лучше воспользоваться `медианой` т.к. помимо пропусков, в данных есть аномальные значения, о которых я расскажу ниже:

**Проверим максимальное значение стажа:**

In [11]:
data['days_employed'].max()

401755.40047533

Получается, кто-то работает `1100 года`, а говорят бессмертных не существует.
Т.к. **в данных отсутствует точная информация о стаже**, а в трудовой стаж, дающий право на отпуск, включаются все время, когда за сотрудником сохранялось место работы (должность), в том числе выходные и праздничные дни, дни отпуска, больничного и т п., то сделаем функцию:

In [12]:
def max_seniority(value):
    current_seniority = value['days_employed']
    if value['gender'] == 'M' and value['dob_years'] > 60 : 
        max_years = 60
    elif value['gender'] == 'F' and value['dob_years'] > 55:
        max_years = 55
    else: 
        max_years = value['dob_years']

        max_seniority = (max_years - 18) * 365
        if current_seniority > max_seniority:
            return max_seniority
        elif current_seniority < 0:
            return (current_seniority * -1)
        else:
            return current_seniority
data['days_employed'] = data.apply(max_seniority, axis = 1)

In [13]:
display(data['days_employed'].max())

15330.0

**Ключевые проблемы появления пропусков в dataset:**
* Самая очевидная проблема: человеческий фактор.
* При перезаписи/переносе файла между людьми/дисками и.т.п данные могут повредиться

**Проверим кол-во пропусков в столбцах. А также, заменим вещественный тип данных в столбце total_income на целочисленный(int64).**

In [14]:
data.isnull().sum()

children               0
days_employed       3798
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

In [15]:
data['total_income'] = data['total_income'].astype('int64')

### Обработка/анализ уникальных значений

**Снизу выведены столбцы с уникальными значениями без артефактов/пропусков:**

In [16]:
display('Тип занятости:', data['income_type'].unique())
display('Идентификатор семейного положения:', data['family_status_id'].unique())
display('Задолженность:', data['debt'].unique())

'Тип занятости:'

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

'Идентификатор семейного положения:'

array([0, 1, 2, 3, 4])

'Задолженность:'

array([0, 1])

**Приведем все значения столбца `семейный статус` к нижнему регистру**

In [17]:
data['family_status'] = data['family_status'].str.lower()

In [18]:
display('Семейное положение:', data['family_status'].unique())

'Семейное положение:'

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

**Выведем на экран уникальные значения столбца** `children` **методом** `.unique()`

In [19]:
display('Кол-во детей:', data['children'].unique())

'Кол-во детей:'

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**Видим 2 настораживающих значения:** `-1` и `20`

Заменим значение:
* `-1` на `1`
* `20` на `2`

In [20]:
data.loc[data['children'] == -1, 'children'] = 1
data.loc[data['children'] == 20, 'children'] = 2
display('Кол-во детей:', data['children'].unique())

'Кол-во детей:'

array([1, 0, 3, 2, 4, 5])

**Двигаемся дальше и выведем на экран столбец** `education`

In [21]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

**Видим неявные дубликаты, приведем все значению к 1 виду:**

In [22]:
data['education'] = data['education'].str.lower()
display(data['education'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

**Выведем на экран столбец** `gender`

In [23]:
display('Пол:', data['gender'].value_counts())

'Пол:'

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Видим 1 значение `XNA`:
    
**Так как значение 1 и не сможет повлиять на дальнейший анализ, удалим данную строчку.**

In [24]:
data = data[data['gender'] != 'XNA']

Перейдем к столбцу с `возрастом`

In [25]:
display('Возраст:', data['dob_years'].value_counts())

'Возраст:'

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    263
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

**Смотрим глубже**

In [26]:
display(data.loc[data['dob_years'] == 0])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,-6570.0,0,среднее,1,женат / замужем,0,F,пенсионер,0,71291,автомобиль
149,0,-6570.0,0,среднее,1,в разводе,3,F,сотрудник,0,70176,операции с жильем
270,3,-6570.0,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166,ремонт жилью
578,0,-6570.0,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620,строительство собственной недвижимости
1040,0,-6570.0,0,высшее,0,в разводе,3,F,компаньон,0,303994,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,-6570.0,0,среднее,1,женат / замужем,0,F,сотрудник,0,142594,жилье
20462,0,-6570.0,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193,покупка своего жилья
20577,0,-6570.0,0,среднее,1,не женат / не замужем,4,F,пенсионер,0,129788,недвижимость
21179,2,-6570.0,0,высшее,0,женат / замужем,0,M,компаньон,0,240702,строительство жилой недвижимости


**Заменим значение `0` медианым значением согласно значению столбца** `income_type`

In [27]:
dob_years_median = data.groupby('income_type')['dob_years'].median()
display(dob_years_median)

income_type
безработный        38.0
в декрете          39.0
госслужащий        40.0
компаньон          39.0
пенсионер          60.0
предприниматель    42.5
сотрудник          39.0
студент            22.0
Name: dob_years, dtype: float64

In [28]:
data.loc[(data['income_type'] == 'безработный ') & (data['dob_years'] == 0), 'dob_years'] = dob_years_median[0]
data.loc[(data['income_type'] == 'в декрете') & (data['dob_years'] == 0), 'dob_years'] = dob_years_median[1]
data.loc[(data['income_type'] == 'госслужащий') & (data['dob_years'] == 0), 'dob_years'] = dob_years_median[2]
data.loc[(data['income_type'] == 'компаньон') & (data['dob_years'] == 0), 'dob_years'] = dob_years_median[3]
data.loc[(data['income_type'] == 'пенсионер') & (data['dob_years'] == 0), 'dob_years'] = dob_years_median[4]
data.loc[(data['income_type'] == 'предприниматель') & (data['dob_years'] == 0), 'dob_years'] = dob_years_median[5]
data.loc[(data['income_type'] == 'сотрудник') & (data['dob_years'] == 0), 'dob_years'] = dob_years_median[6]
data.loc[(data['income_type'] == 'студент') & (data['dob_years'] == 0), 'dob_years'] = dob_years_median[7]

**Проверим таблицу на** `0`

In [29]:
display(data.loc[data['dob_years'] == 0])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


### Группировка столбца `total_income` по категориям:

**На основании диапазонов, указанных заказчиком, создадим столбец `total_income_category` с категориями:**

In [30]:
def income_category(income):
    if income < 30000:
        return 'E'
    if 30001 <= income <= 50000:
        return 'D'
    if 50001 <= income <= 200000:
        return 'C'
    if 200001 <= income <= 1000000:
        return 'B'
    if income >= 1000001:
        return 'A'
    return 'группа не определена'
data['total_income_category'] = data['total_income'].apply(income_category)
display(data['total_income_category'].value_counts())

C    16086
B     5041
D      350
A       25
E       22
Name: total_income_category, dtype: int64

### Создание 2 новых dataframe со столбцами:
* **education_id и education — в первом;**
* **family_status_id и family_status — во втором.**

In [31]:
education_id = [0, 1, 2, 3, 4]
education = ['высшее', 'начальное', 'неоконченное высшее', 'среднее', 'ученая степень']
education_df = dict(zip(education_id, education))
display(education_df)

{0: 'высшее',
 1: 'начальное',
 2: 'неоконченное высшее',
 3: 'среднее',
 4: 'ученая степень'}

In [32]:
family_status_id = [0, 1, 2, 3, 4]
family_status = ['женат / замужем', 'гражданский брак', 'вдовец / вдова', 'в разводе', 'Не женат / не замужем']
family_df = dict(zip(family_status_id, family_status))
display(family_df)

{0: 'женат / замужем',
 1: 'гражданский брак',
 2: 'вдовец / вдова',
 3: 'в разводе',
 4: 'Не женат / не замужем'}

In [33]:
data.drop(["education", "family_status"], axis = 1, inplace = True)

## **Лемматизация**

**Проверим и обработаем столбец** `purpose`

In [34]:
m = Mystem()
def purpose_group (purpose):
    lemmas = m.lemmatize(purpose)
    for i in lemmas:
        if 'свадьб' in i:
            return 'проведение свадьбы'
        elif 'образов' in i:
            return 'получение образования'
        elif 'недвиж' in i:
            return 'операции с недвижимостью'
        elif 'жил' in i:
            return 'операции с недвижимостью'
        elif 'автомобил' in i:
            return 'операции с автомобилем'

In [35]:
data['purpose_category'] = data['purpose'].apply(purpose_group)
data['purpose_category'].value_counts()

операции с недвижимостью    10839
операции с автомобилем       4315
получение образования        4022
проведение свадьбы           2348
Name: purpose_category, dtype: int64

**После создания 4 категорий из столбца `purpose`, мы можем заметить, что большая доля кредитов - операции с `недвижимостью`.**

## **Ответьты на вопросы**

### Присутствует ли зависимость между количеством детей и возвратом кредита в срок?

**Посчитаем общую долю просроченных платажей:**

In [36]:
overdue_p = data[(data['debt'] == 1)].count() / data[(data['debt'] == 0)].count()
display('Шанс просроченного платежа состовляет {:.1%}'.format((overdue_p['debt'])))

'Шанс просроченного платежа состовляет 8.8%'

**Построим сводную таблицу и посмотрим количество просрочек:**

In [37]:
children_debt = data.pivot_table(index=['children'], columns='debt', values = 'family_status_id', aggfunc='count', fill_value=0)
display(children_debt)

debt,0,1
children,,
0,13085,1063
1,4420,445
2,1929,202
3,303,27
4,37,4
5,9,0


In [38]:
children_debt['% non payments'] = (children_debt[1] * 100  / children_debt[0])

In [39]:
display(children_debt)

debt,0,1,% non payments
children,,,
0,13085,1063,8.123806
1,4420,445,10.067873
2,1929,202,10.471747
3,303,27,8.910891
4,37,4,10.810811
5,9,0,0.000000


**Вывод:**
Клиенты с количеством детей: 0-2 в большей степени подвержены неоплате кредита в срок, чем клиенты с 3-4 детьми. Также можно заметить, что клиенты с 5 детьми совсем не имеют задолженностей! 

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [40]:
family_debt = data.pivot_table(index='family_status_id', columns='debt', values='dob_years', aggfunc='count',fill_value=0)
family_debt['% non payments'] = (family_debt[1] * 100  / family_debt[0])
display(family_debt)
display(family_df)

debt,0,1,% non payments
family_status_id,,,
0,11449,931,8.131715
1,3788,388,10.242872
2,897,63,7.023411
3,1110,85,7.657658
4,2539,274,10.791650


{0: 'женат / замужем',
 1: 'гражданский брак',
 2: 'вдовец / вдова',
 3: 'в разводе',
 4: 'Не женат / не замужем'}

**Вывод:**

`Не женатые/не замужние` & люди состоящие в `гражданском браке` с большей вероятностью не вернут кредит в срок.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [41]:
income_debt = data.pivot_table(index=['total_income_category'], columns='debt', values='total_income', aggfunc='count')
income_debt['% non payments'] = (income_debt[1] * 100  / income_debt[0])
display(income_debt)

debt,0,1,% non payments
total_income_category,,,
A,23,2,8.695652
B,4685,356,7.598719
C,14726,1360,9.235366
D,329,21,6.382979
E,20,2,10.000000


**Вывод:**

Люди с более низким уровнем дохода `категория Е` имеет большую долю просроченных платещей, нежели с высоким уровнем дохода `категория А`.

Также, стоит отметить, долю просроченных платежей среднего класса. `категория С`

### Как разные цели кредита влияют на его возврат в срок?

In [42]:
purpose_debt = data.pivot_table(index=['purpose_category'], columns='debt', values='purpose', aggfunc='count')
purpose_debt['% non payments'] = (purpose_debt[1] * 100  / purpose_debt[0])
display(purpose_debt)

debt,0,1,% non payments
purpose_category,,,
операции с автомобилем,3912,403,10.301636
операции с недвижимостью,10057,782,7.775679
получение образования,3652,370,10.131435
проведение свадьбы,2162,186,8.603145


**Вывод:**

Чаще всего платят в срок люди с категорией `Операции с недвижимостью`, чаще просрочка у категорий `Операции с автомобилем` и `Получение образования`. Кредиты с категорией `Проведение свадьбы` просрачивают реже, примерно как и в случае недвижимости.

<img src="https://hsto.org/r/w1560/getpro/habr/upload_files/dee/1a0/71f/dee1a071f6203a7c34c0b5b6283bcf83.png" width="800"/>

## **Общий вывод:**

После заполнения всех пропусков, обработке уникальных значений и удаление неявных дубликатов, а также использования сводных таблиц сделаем вывод:

- Количество детей не сильно влияют на сроки выплаты кредита.
- `Не женатые/не замужние`и люди состоящие в `гражданском браке` с большей вероятностью не вернут кредит в срок.
- Люди с доходом `0–30000` и `50001–200000` более склоны к не возвращению кредита в срок.
- Кредиты с целью образования или проведения свадьбы с большей вероятностью будут просрочены.

**Мы ответили на 4 вопроса и установили:**

- Зависимость количества детей и возвратом кредита в срок: Клиенты с количеством детей: 0-2 в большей степени подвержены неоплате кредита в срок, чем клиенты с 3-4 детьми. Также можно заметить, что клиенты с 5 детьми совсем не имеют задолженностей! 


- Расчеты показали, что `не женатые/не замужние` & `люди состоящие в гражданском браке` с большей вероятностью не вернут кредит в срок. Доля клиентов с  просроченными платежами составила: `10.79%` & `10.24%` соответственно.


- Люди с более низкими доходами **(< 30.000)** более склоны к не возвратом кредита в срок, нежели клиенты с  более высоким уровнем дохода. Также, стоит отметить, долю просроченных платежей `категории С` с доходами **(50.000–200.000)**.


- Чаще всего платят в срок люди с категориями `Операции с недвижимостью` и `Проведение свадьбы` , а у категорий `Операции с автомобилем` и `Получение образования` заметна более высокая доля просроченных платещей по кредиту.

- Исходя из проведенного анализа, можно заметить, что многодетные клиенты состоящие в браке, заработком выше среднего самые кредитоспособные для любой категории кредита.